The contents of this course including lectures, labs, homework assignments, and exams have all been adapted from the [Data 8 course at University California Berkley](https://data.berkeley.edu/education/courses/data-8). Through their generosity and passion for undergraduate education, the Data 8 community at Berkley has opened their content and expertise for other universities to adapt in the name of undergraduate education.

In [ ]:
#!pip install datascience
from datascience import *
import numpy as np

from IPython.display import display, Math, Latex

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Chapter 16: Inference for Regression

Thus far, our analysis of the relation between variables has been purely descriptive. We know how to find the best straight line to draw through a scatter plot. The line is the best in the sense that it has the smallest mean squared error of estimation among all straight lines.

* What if our data were only a sample from a larger population? 
* If in the sample we found a linear relation between the two variables, would the same be true for the population? 
* Would it be exactly the same linear relation? 
* Could we predict the response of a new individual who is not in our sample?

If we believe that a sample reflects the underlying population we can infer and predict information about the underlying population.

Our simulations show that if the regression model holds and the sample size is large, then the regression line is likely to be close to the true line. This allows us to estimate the slope of the true line.

We will use our familiar sample of mothers and their newborn babies to develop a method of estimating the slope of the true line.

In [ ]:
baby = Table.read_table('baby.csv')
baby

In [ ]:
def standard_units(any_numbers):
    "Convert any array of numbers to standard units."
    return (any_numbers - np.mean(any_numbers))/np.std(any_numbers)  

def correlation(t, label_x, label_y):
    return np.mean(standard_units(t.column(label_x))*standard_units(t.column(label_y)))

def slope(t, label_x, label_y):
    r = correlation(t, label_x, label_y)
    return r*np.std(t.column(label_y))/np.std(t.column(label_x))

def intercept(t, label_x, label_y):
    return np.mean(t.column(label_y)) - slope(t, label_x, label_y)*np.mean(t.column(label_x))

In [ ]:
baby.scatter('Gestational Days', 'Birth Weight', fit_line=True)

In [ ]:
correlation(baby, 'Gestational Days', 'Birth Weight')

In [ ]:
slope(baby, 'Gestational Days', 'Birth Weight')

But had the scatter plot come out differently, the regression line would have been different and might have had a different slope. How do we figure out how different the slope might have been?

We need another sample of points, so that we can draw the regression line through the new scatter plot and find its slope. But from where will get another sample?

You have guessed it – we will bootstrap our original sample. That will give us a bootstrapped scatter plot, through which we can draw a regression line.

Here is the original scatter diagram from the sample, and four replications of the bootstrap resampling procedure. Notice how the resampled scatter plots are in general a little more sparse than the original. That is because some of the original points do not get selected in the samples.

In [ ]:
def one_bootstrap():
    resample = baby.sample()
    return resample

In [ ]:
baby.scatter('Gestational Days', 'Birth Weight')

In [ ]:
one_bootstrap().scatter('Gestational Days', 'Birth Weight')

In [ ]:
one_bootstrap().scatter('Gestational Days', 'Birth Weight')

In [ ]:
one_bootstrap().scatter('Gestational Days', 'Birth Weight')

In [ ]:
one_bootstrap().scatter('Gestational Days', 'Birth Weight')

In [ ]:
slopes = make_array()
for i in np.arange(5000):
    bootstrap_sample = baby.sample()
    bootstrap_slope = slope(bootstrap_sample, 'Gestational Days', 'Birth Weight')
    slopes = np.append(slopes, bootstrap_slope)
Table().with_column('Bootstrap Slopes', slopes).hist(bins=20)

We can then construct an approximate 95% confidence interval for the slope of the true line, using the bootstrap percentile method. The confidence interval extends from the 2.5th percentile to the 97.5th percentile of the 5000 bootstrapped slopes.

In [ ]:
left = percentile(2.5, slopes)
right = percentile(97.5, slopes)
left, right

Let's wrap this into one function.

In [ ]:
def bootstrap_slope(table, x, y, repetitions):
    
    # For each repetition:
    # Bootstrap the scatter, get the slope of the regression line,
    # augment the list of generated slopes
    slopes = make_array()
    for i in np.arange(repetitions):
        bootstrap_sample = table.sample()
        bootstrap_slope = slope(bootstrap_sample, x, y)
        slopes = np.append(slopes, bootstrap_slope)
    
    # Find the endpoints of the 95% confidence interval for the true slope
    left = percentile(2.5, slopes)
    right = percentile(97.5, slopes)
    
    # Slope of the regression line from the original sample
    observed_slope = slope(table, x, y)
    
    # Display results
    Table().with_column('Bootstrap Slopes', slopes).hist(bins=20)
    plots.plot(make_array(left, right), make_array(0, 0), color='yellow', lw=8);
    print('Slope of regression line:', observed_slope)
    print('Approximate 95%-confidence interval for the true slope:')
    print(left, right)

In [ ]:
bootstrap_slope(baby, 'Gestational Days', 'Birth Weight', 5000)

What is the slope is zero and there is no correlation between the x and y variables?

In [ ]:
baby.scatter('Maternal Age', 'Birth Weight')

In [ ]:
slope(baby, 'Maternal Age', 'Birth Weight')

We can come up with a hypothsis test:

**Null Hypothesis:** The slope of the true line is 0.

**Alternative Hypothesis:** The slope of the true line is not 0.

In [ ]:
bootstrap_slope(baby, 'Maternal Age', 'Birth Weight', 5000)

Because the interval contains 0, we cannot reject the null hypothesis that the slope of the true linear relation between maternal age and baby’s birth weight is 0. Based on this analysis, it would be unwise to predict birth weight based on the regression model with maternal age as the predictor.

We are trying to make predictions on $y$ given a new value of $x$

In [ ]:
def fitted_value(table, x, y, given_x):
    a = slope(table, x, y)
    b = intercept(table, x, y)
    return a * given_x  + b

In [ ]:
fit_300 = fitted_value(baby, 'Gestational Days', 'Birth Weight', 300)
fit_300

Given a value of 300 gestation days we will predict that the babys weight will be about 129.2 oucnes.  But how does our prediction of $x$ change if the regression line was different?

Let us define a function called bootstrap_prediction to do this. The function takes five arguments:

   * the name of the table

   * the column labels of the predictor and response variables, in that order

   * the value of 

   * at which to make the prediction

   * the desired number of bootstrap repetitions

In each repetition, the function bootstraps the original scatter plot and finds the predicted value of $y$ based on the specified value of $x$. Specifically, it calls the function fitted_value that we defined earlier in this section to find the fitted value at the specified $x$.

Finally, it draws the empirical histogram of all the predicted values, and prints the interval consisting of the “middle 95%” of the predicted values. It also prints the predicted value based on the regression line through the original scatter plot.

In [ ]:
# Bootstrap prediction of variable y at new_x
# Data contained in table; prediction by regression of y based on x
# repetitions = number of bootstrap replications of the original scatter plot

def bootstrap_prediction(table, x, y, new_x, repetitions):
    
    # For each repetition:
    # Bootstrap the scatter; 
    # get the regression prediction at new_x; 
    # augment the predictions list
    predictions = make_array()
    for i in np.arange(repetitions):
        bootstrap_sample = table.sample()
        bootstrap_prediction = fitted_value(bootstrap_sample, x, y, new_x)
        predictions = np.append(predictions, bootstrap_prediction)
        
    # Find the ends of the approximate 95% prediction interval
    left = percentile(2.5, predictions)
    right = percentile(97.5, predictions)
    
    # Prediction based on original sample
    original = fitted_value(table, x, y, new_x)
    
    # Display results
    Table().with_column('Prediction', predictions).hist(bins=20)
    plots.xlabel('predictions at x='+str(new_x))
    plots.plot(make_array(left, right), make_array(0, 0), color='yellow', lw=8);
    print('Height of regression line at x='+str(new_x)+':', original)
    print('Approximate 95%-confidence interval:')
    print(left, right)

In [ ]:
bootstrap_prediction(baby, 'Gestational Days', 'Birth Weight', 300, 5000)

The interval ranges from about 127 to about 131. The prediction based on the original sample was about 129, which is close to the center of the interval.

Let's predict a baby weight with the mean gestaion days.

In [ ]:
np.mean(baby.column('Gestational Days'))

In [ ]:
bootstrap_prediction(baby, 'Gestational Days', 'Birth Weight', np.mean(baby.column('Gestational Days')), 5000)

Notice that this interval is narrower than the prediction interval at 300 gestational days. Let us investigate the reason for this.

So 279 is closer to the center of the distribution than 300 is. Typically, the regression lines based on the bootstrap samples are closer to each other near the center of the distribution of the predictor variable. Therefore all of the predicted values are closer together as well. This explains the narrower width of the prediction interval.

In [ ]:
baby.scatter('Gestational Days', 'Birth Weight', fit_line=True)